In [5]:
%pip install -q --disable-pip-version-check \
    evaluate==0.4.0 \
    py7zr==0.20.4 \
    sentencepiece==0.1.99 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.4.0 \
    trl==0.7.2
%pip install -q    wandb bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 718.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import output
output.enable_custom_widget_manager()

In [3]:
PROJECT = "FlanT5-Lora"
MODEL_NAME = 'google/flan-t5-base'
DATASET = "knkarthick/dialogsum"

In [6]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

In [7]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [10]:
!ls

quantitative_results	 test_cooccur_old.csv	   train.jsonl
results_qualitative	 test.jsonl		   validate.jsonl
test_cooccur_new.csv	 train_cooccur_new.csv	   validation_cooccur_new.csv
test_cooccur_new.gsheet  train_cooccur_new.gsheet  validation_cooccur_new.gsheet


In [9]:
import os
os.chdir('drive/MyDrive/nlp_proj_dataset')

In [11]:
from datasets import load_dataset

# Define the paths to your files

def create_hf_dataset():
    data_files = {}
    data_files["train"] = 'train.jsonl'
    data_files["test"]  = 'test.jsonl'
    data_files["validation"] = 'validate.jsonl'

    # Load the datasets
    dataset = load_dataset('json', data_files=data_files)

    # Print the first example of the training dataset
    print(dataset)

    return dataset

# check the number of examples in each dataset

dataset = create_hf_dataset()

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'inputs', 'target'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['idx', 'inputs', 'target'],
        num_rows: 150
    })
    validation: Dataset({
        features: ['idx', 'inputs', 'target'],
        num_rows: 50
    })
})


In [12]:
dataset['train'].to_pandas()

,idx,inputs,target
0,0,SUBJECT: who and where to get cetirizine - D M...,Who manufactures cetirizine?
1,1,who makes bromocriptine i am wondering what co...,Who manufactures bromocriptine?
2,2,SUBJECT: nulytely MESSAGE: Hello can you tell ...,"Who makes nulytely, and where can I buy it?"
3,3,Williams' syndrome I would like to have my dau...,Where can I get genetic testing for william's ...
4,4,ClinicalTrials.gov - Question - general inform...,Where can I get genetic testing for multiple m...
...,...,...,...
995,995,SUBJECT: after surgery of ear drum still same ...,What are the treatments for perforated eardrum?
996,996,SUBJECT: ClinicalTrials.gov - Question - speci...,What are the treatments for Glycogen storage d...
997,997,MESSAGE: I have numbness/tingling in my lower ...,Where can I find information and treatment for...
998,998,SUBJECT: sleep apnea MESSAGE: I was diagnosed ...,How long does swelling from sleep apnea take t...


Modify dataset with co-occurrence

In [13]:
train_set = dataset['train'].to_pandas()
print(train_set)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
print(stop_words)
#print(tuydhtd)
all_words = []
for index, row in train_set.iterrows():
  idx = row['idx']
  inputs = row['inputs']
  target = row['target']

  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

  # Removing punctuations in string
  # Using loop + punctuation string
  for ele in inputs:
      if ele in punc:
          inputs = inputs.replace(ele, "")

  word_tokens = word_tokenize(inputs)
  # converts the words in word_tokens to lower case and then checks whether
  #they are present in stop_words or not
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  #with no lower case conversion
  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)

  #print(word_tokens)
  #print(filtered_sentence)
  for eachword in filtered_sentence:
    if eachword.lower() not in all_words:
      all_words.append(eachword.lower())
  #break

print(all_words)
print(len(all_words))

     idx                                             inputs  \
0      0  SUBJECT: who and where to get cetirizine - D M...   
1      1  who makes bromocriptine i am wondering what co...   
2      2  SUBJECT: nulytely MESSAGE: Hello can you tell ...   
3      3  Williams' syndrome I would like to have my dau...   
4      4  ClinicalTrials.gov - Question - general inform...   
..   ...                                                ...   
995  995  SUBJECT: after surgery of ear drum still same ...   
996  996  SUBJECT: ClinicalTrials.gov - Question - speci...   
997  997  MESSAGE: I have numbness/tingling in my lower ...   
998  998  SUBJECT: sleep apnea MESSAGE: I was diagnosed ...   
999  999  SUBJECT: Home resources for AMD people MESSAGE...   

                                                target  
0                         Who manufactures cetirizine?  
1                      Who manufactures bromocriptine?  
2          Who makes nulytely, and where can I buy it?  
3    Where can 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


{'about', 'd', 'more', 'not', 'most', 'through', 'why', 'when', 'for', 'there', "she's", "you'd", 'the', 'wouldn', 'they', 'is', "it's", 't', 'yourself', "hasn't", 'if', 'your', 'y', 'has', 'here', 'them', 'of', 'over', 'with', 'out', 'shouldn', 'wasn', 'had', 'it', 'again', 'hers', 'above', 'than', "mightn't", 'doing', 'should', 'ours', "shan't", "shouldn't", 've', 'an', 'm', 'such', 'then', 'which', 'were', 'her', 'and', 'at', 'couldn', 'you', "you're", "isn't", 'any', 'to', 'ma', "doesn't", 'a', 'other', 'll', "should've", 'until', 'he', 'o', 'now', 'same', 'whom', 'those', 'into', 'as', 'up', "needn't", 'between', 'under', 's', 'ain', 'we', 'won', 'in', 'just', "you've", 'she', 'because', 'off', 'down', 'its', 'where', 'was', 'being', 'himself', "mustn't", 'how', 'all', 'his', 'that', 'these', 'own', 'be', 'so', 'been', 're', "didn't", 'yourselves', 'aren', 'before', 'i', 'didn', "wouldn't", 'only', 'having', 'against', "aren't", 'themselves', 'mustn', 'both', 'this', 'by', 'some',

In [14]:
nltk.download('averaged_perceptron_tagger')
tagged = nltk.pos_tag(all_words)
acceptable=['NN','NNP','NNS']
for eachtag in tagged:
  wordfocus = eachtag[0]
  pos = eachtag[1]
  if pos not in acceptable:
    all_words.remove(wordfocus)
print(len(all_words))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


3395


In [15]:
n = 3395
co_mat = np.zeros((n,n))
co_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
window_len = 3
uniq_wrds = all_words
def update_co_mat(x):
    # Get all the words in the sentence and store it in an array wrd_lst
    wrd_list = x.split(' ')

    # Consider each word as a focus word
    for focus_wrd_indx, focus_wrd in enumerate(wrd_list):
        focus_wrd = focus_wrd.lower()
        # Get the indices of all the context words for the given focus word
        for contxt_wrd_indx in range((max(0,focus_wrd_indx - window_len)),(min(len(wrd_list),focus_wrd_indx + window_len +1))):
            # If context words are in the unique words list
            if wrd_list[contxt_wrd_indx] in uniq_wrds:
                try:
                  # To identify the row number, get the index of the focus_wrd in the uniq_wrds list
                  co_mat_row_indx = uniq_wrds.index(focus_wrd)
                except:
                  pass

                try:
                  # To identify the column number, get the index of the context words in the uniq_wrds list
                  co_mat_col_indx = uniq_wrds.index(wrd_list[contxt_wrd_indx])
                except:
                  pass

                # Update the respective columns of the corresponding focus word row
                try:
                  co_mat[co_mat_row_indx][co_mat_col_indx] += 1
                except:
                  pass
    return co_mat
from tqdm import tqdm
for index,row in tqdm(train_set.iterrows()):
  idx = row['idx']
  inputs = row['inputs']
  target = row['target']
  co_mat = update_co_mat(inputs)
print(co_mat)

1000it [00:30, 33.30it/s]

[[  15.    0.    2. ...    0.    0.    0.]
 [   3. 1234.  278. ...    0.    0.    0.]
 [   3.   42.  691. ...    0.    0.    0.]
 ...
 [   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]]


In [17]:
maxval = np.amax(co_mat)
minval = np.amin(co_mat)
print(maxval)
print(minval)
print(np.mean(co_mat))

1234.0
0.0
0.005561501037868649


In [18]:
threshold = 10
for index, row in tqdm(train_set.iterrows()):
  added_str = ""
  inputs = row['inputs']
  target = row['target']
  idx = row['idx']
  words = inputs.split()
  for eachword in words:
    if eachword in all_words:
      index_word = all_words.index(eachword)
      for j in range(index_word,n):
        if j!=index_word:
          cell_val = co_mat[index_word][j]
          if cell_val > threshold:
            if all_words[j] in words:
              #print("came here")
              added_str += " <" + eachword + "-" + all_words[j] + ">"
  inputs += added_str
  #print(inputs)
  #break
  #row['inputs'] = inputs
  train_set.at[index, 'inputs'] = inputs
print(train_set)


1000it [00:23, 43.20it/s]

     idx                                             inputs  \
0      0  SUBJECT: who and where to get cetirizine - D M...   
1      1  who makes bromocriptine i am wondering what co...   
2      2  SUBJECT: nulytely MESSAGE: Hello can you tell ...   
3      3  Williams' syndrome I would like to have my dau...   
4      4  ClinicalTrials.gov - Question - general inform...   
..   ...                                                ...   
995  995  SUBJECT: after surgery of ear drum still same ...   
996  996  SUBJECT: ClinicalTrials.gov - Question - speci...   
997  997  MESSAGE: I have numbness/tingling in my lower ...   
998  998  SUBJECT: sleep apnea MESSAGE: I was diagnosed ...   
999  999  SUBJECT: Home resources for AMD people MESSAGE...   

                                                target  
0                         Who manufactures cetirizine?  
1                      Who manufactures bromocriptine?  
2          Who makes nulytely, and where can I buy it?  
3    Where can 

In [ ]:
train_set.to_csv('random_csv.csv')